# Expirence 4

# 0. Introduction

## Differential Equations

TODO

In [25]:
import math
import numpy as np
from scipy.optimize import fsolve

# Analysis and plotting modules
import pandas as pd
import plotly
import plotly.express as px

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor


pd.options.plotting.backend = "plotly"

## 3.a

In [11]:
df = pd.read_csv('data/teste_3600-ciclos-20.txt', delimiter = "\t", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,0,0,0,0,0,0,0,25.433,0,...,0,0,0,0,0,0,0,0,0,3600
1,0,0,0,0,0,5,0,0,25.482,0,...,0,0,0,0,0,0,0,0,0,3600
2,0,0,0,0,0,10,0,0,25.427,0,...,0,0,0,0,0,0,0,0,0,3600
3,0,0,0,0,0,15,0,0,25.807,0,...,0,0,0,0,0,0,0,0,0,3600
4,0,0,0,0,0,20,0,0,25.994,0,...,0,0,0,0,0,0,0,0,0,3600


In [28]:
cols_dict = {
    5:'time (s)',
    8:'Tc',
    19:'Ti',
    24:'Te',
    31:'Ta',
    41:'Freq'
}

df.rename(columns=cols_dict, inplace=True)
df = df[cols_dict.values()]
df[['Ta', 'Ti', 'Te', 'Tc']] = df[['Ta', 'Ti', 'Te', 'Tc']]+273
df['S'] = df['Freq'].apply(lambda x: 1 if x>0 else 0)
df

/home/yves/.cache/pypoetry/virtualenvs/process-simulation-6FL6Grl8-py3.7/lib/python3.7/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,time (s),Tc,Ti,Te,Ta,Freq,S
0,0,298.433,255.065,260.928,292.859,3600,1
1,5,298.482,255.109,260.847,292.840,3600,1
2,10,298.427,255.119,260.557,292.889,3600,1
3,15,298.807,255.152,259.422,292.823,3600,1
4,20,298.994,255.158,258.853,292.861,3600,1
...,...,...,...,...,...,...,...
1016,5080,298.203,254.956,260.968,292.865,0,0
1017,5085,298.252,254.972,261.008,292.842,0,0
1018,5090,298.214,254.996,261.049,292.840,0,0
1019,5095,298.171,255.058,260.968,292.857,0,0


In [29]:
fig = px.line(df, x='time (s)', y='S')
fig.show()

In [32]:
fig = px.line(df, x='time (s)', y='Freq')
fig.show()

In [30]:
fig = px.line(df, x='time (s)', y=['Ta', 'Ti', 'Te', 'Tc'])
fig.show()

## 3.b

In [33]:
df

,time (s),Tc,Ti,Te,Ta,Freq,S
0,0,298.433,255.065,260.928,292.859,3600,1
1,5,298.482,255.109,260.847,292.840,3600,1
2,10,298.427,255.119,260.557,292.889,3600,1
3,15,298.807,255.152,259.422,292.823,3600,1
4,20,298.994,255.158,258.853,292.861,3600,1
...,...,...,...,...,...,...,...
1016,5080,298.203,254.956,260.968,292.865,0,0
1017,5085,298.252,254.972,261.008,292.842,0,0
1018,5090,298.214,254.996,261.049,292.840,0,0
1019,5095,298.171,255.058,260.968,292.857,0,0


In [39]:
X = df[['Tc', 'Ti', 'Te', 'Ta', 'S']][:-1]
X

,Tc,Ti,Te,Ta,S
0,298.433,255.065,260.928,292.859,1
1,298.482,255.109,260.847,292.840,1
2,298.427,255.119,260.557,292.889,1
3,298.807,255.152,259.422,292.823,1
4,298.994,255.158,258.853,292.861,1
...,...,...,...,...,...
1015,298.220,254.903,260.968,292.893,0
1016,298.203,254.956,260.968,292.865,0
1017,298.252,254.972,261.008,292.842,0
1018,298.214,254.996,261.049,292.840,0


In [40]:
Y = df[['Tc', 'Ti', 'Te']].shift()[1:]
Y

,Tc,Ti,Te
1,298.433,255.065,260.928
2,298.482,255.109,260.847
3,298.427,255.119,260.557
4,298.807,255.152,259.422
5,298.994,255.158,258.853
...,...,...,...
1016,298.220,254.903,260.968
1017,298.203,254.956,260.968
1018,298.252,254.972,261.008
1019,298.214,254.996,261.049


In [42]:
from sklearn.linear_model import LinearRegression

In [43]:
lm_model = LinearRegression()

In [44]:
lm_model.fit(X,Y)

LinearRegression()

In [45]:
lm_model.coef_

array([[ 1.00000000e+00,  2.60065168e-16,  2.46076380e-16,
        -1.09690038e-14, -4.11856997e-15],
       [-3.08395255e-16,  1.00000000e+00, -3.33066907e-16,
         5.71374545e-16, -3.19189120e-16],
       [ 6.14642406e-16, -3.33066907e-16,  1.00000000e+00,
        -4.80409983e-16, -1.80411242e-16]])

In [57]:
Y_pred = lm_model.predict(X)

results = Y.copy()

results[['Tc_pred', 'Ti_pred', 'Te_pred']] = Y_pred
results['time'] = df['time (s)'][1:]
results.head()

,Tc,Ti,Te,Tc_pred,Ti_pred,Te_pred,time
1,298.433,255.065,260.928,298.433,255.065,260.928,5
2,298.482,255.109,260.847,298.482,255.109,260.847,10
3,298.427,255.119,260.557,298.427,255.119,260.557,15
4,298.807,255.152,259.422,298.807,255.152,259.422,20
5,298.994,255.158,258.853,298.994,255.158,258.853,25
